In [27]:
import geopandas as gpd
import pandas as pd
import requests
import folium
import time

from shapely.geometry import Point

In [28]:
customer_transaction = r"C:/GD_2025/6_spatial_data_science/1_location_fnb/2_pipeline/location_intelligence_potential_sales_areas/data/row-data/1_input_datas/2_customer_trasaction/restaurant_dataset_transactions.csv"
geojson_jakarta = r"C:/GD_2025/6_spatial_data_science/1_location_fnb/2_pipeline/location_intelligence_potential_sales_areas/data/vektor/jakarta.geojson"

In [29]:
# read customer transaction data
dateframe_customer_transaction = pd.read_csv(customer_transaction)
dateframe_customer_transaction_cleanned = dateframe_customer_transaction.drop(columns=["badan_usaha", "penanggung_jawab", "wilayah"])
display(dateframe_customer_transaction_cleanned)

# read jakarta areas data
geodataframe_jakarta = gpd.read_file(geojson_jakarta)
minx, miny, maxx, maxy = geodataframe_jakarta.total_bounds

print(f"bounding box Jakarta: {miny}, {minx}, {maxy}, {maxx})")

,nama_usaha,alamat_usaha,jenis_usaha
0,1001 Entertainment Club,"Jl. Kunir No. 7, Pinangsia",Restoran
1,9 Square Pool & Resto,Jl. Anggrek Cakra No. 7 RT004/RW006,Restoran
2,99,Jl. Mangga Besar Raya No. 2B,Restoran
3,A & W Family,"Jl. Kamal Raya Outer Ring Road, Cengkareng",Restoran
4,A & W Family,Jl. Pancoran No. 23,Restoran
...,...,...,...
3952,Yakoya,Jl. Raya Boulevard Barat Blok LC 6 No. 16-18,Restoran
3953,Yamagoya Ramen,"Mall Of Indonesia, Jl. Boulevard Barat Raya No. 1",Restoran
3954,Yoshinoya,Jl. Raya Boulevard Blok XC No. 7,Restoran
3955,Yuraku,Mal Kelapa Gading I Lt. Dasar #195,Restoran


bounding box Jakarta: -6.372625555999946, 106.68580472300005, -6.07410022199997, 106.973391)


In [30]:
def get_coordinates_from_osm(address):
    nominatim_url = "https://nominatim.openstreetmap.org/search"

    params = {
            "q": address,  
            "format": "json",  
            "limit": 1,  
            "addressdetails": 1  
        }

    try:
        response = requests.get(nominatim_url, params=params, headers={"User-Agent": "location"})
        data = response.json()

        if "elements" in data and len(data["elements"]) > 0:
            lat = data["elements"][0]["lat"]
            lon = data["elements"][0]["lon"]
            return lat, lon
        else:
            return None, None
    except Exception as error:
        print(f"Upss, error getting datas: {error}")

In [ ]:
dataframe_with_coordinate = dateframe_customer_transaction_cleanned.copy()

dataframe_with_coordinate["latitude"] = None
dataframe_with_coordinate["longitude"] = None

for index, row in dataframe_with_coordinate.iterrows():
    address = row["alamat_usaha"]

    lat, lon = get_coordinates_from_osm(address)

    dataframe_with_coordinate.at[index, "latitude"] = lat
    dataframe_with_coordinate.at[index, "longitude"] = lon

    time.sleep(1)

display(dataframe_with_coordinate)